In [7]:
import numpy as np #ref
import os
from os import path as op
import string
import re
from subprocess import Popen
import xarray as xr #ref
import metpy #ref
import matplotlib.pyplot as plt #ref
import cartopy.crs as ccrs #ref
import cartopy.feature as cfeature
from pyresample import image, geometry #ref
import seaborn as sns #ref
import netCDF4
sns.set(style="darkgrid")
%matplotlib inline

#sys.path.append('')


ImportError: /scratch/adomakor412/conda/envs/MyEnv/lib/python3.5/site-packages/netCDF4/../../../././libcrypto.so.1.0.0: cannot read file data: Stale file handle

In [ ]:
def create_nc_Numpy(ncFile, pathIn, pathOut):
    myFile = xr.open_dataset(op.join(pathIn,ncFile))
    dat = myFile.metpy.parse_cf('Rad')
    geos = dat.metpy.cartopy_crs

    cartopy_extent_goes = geos.x_limits + geos.y_limits
    pyresample_extent_goes = (cartopy_extent_goes[0],
                                cartopy_extent_goes[2],
                                cartopy_extent_goes[1],
                                cartopy_extent_goes[3])
    goes_params = geos.proj4_params
    rad = dat.data
    
    def normIm(im,gamma=1.0,reverse=False):
        nim = ((im-np.nanmin(im))*(np.nanmax(im)-np.nanmin(im))**(-1))
        if reverse:#want clouds to be white
            nim = (1.0-nim**(gamma))
        return nim
    
    def goes_2_roi(loaded_goes, 
               target_extent,
               target_rows,#actual length or base
               target_cols,#actual width or height
               cartopy_target_proj,
               data_key='Rad',
               radius_of_influence=50000):
        """Function that goes from loaded GOES data to data resampled in a projection for an extent"""
        dat = loaded_goes.metpy.parse_cf('Rad')
        geos_crs = dat.metpy.cartopy_crs
        cartopy_source_extent = geos_crs.x_limits + geos_crs.y_limits
        pyresample_source_extent = (cartopy_source_extent[0],
                                    cartopy_source_extent[2],
                                    cartopy_source_extent[1],
                                    cartopy_source_extent[3])
        rad = dat.data
        source_area = geometry.AreaDefinition('GOES-1X', 'Full Disk','GOES-1X', 
                                              geos_crs.proj4_params,
                                              rad.shape[1], rad.shape[0],
                                              pyresample_source_extent)
        area_target_def = geometry.AreaDefinition('areaTest', 'Target Region', 'areaTest',
                                            cartopy_target_proj.proj4_params,
                                            target_rows, target_cols,
                                            target_extent)
        geos_con_nn = image.ImageContainerNearest(rad, 
                                                source_area, 
                                                radius_of_influence=radius_of_influence)

        # Here we are using pyresample for the remapping
        area_proj_con_nn = geos_con_nn.resample(area_target_def)
        return area_proj_con_nn.image_data
        
    def cartopy_pyresample_toggle_extent(input_extent):
        return np.array(input_extent)[np.array([0,2,1,3])]

    def trasform_cartopy_extent(source_extent,source_proj, target_proj):
        target_extent = target_proj.transform_points(source_proj, 
                                                     np.array(source_extent[:2]),
                                                     np.array(source_extent[2:])).ravel()
        # target_extent in 3D, must be in 2D
        return cartopy_pyresample_toggle_extent(np.array(target_extent)[np.array([0,1,3,4])])
    pc = ccrs.PlateCarree()
    mc = ccrs.Mercator()

    # Convert extent from pc to mc (both cylindrical projections)
    extent_pc = [-109.59326, -102.40674, 8.94659, -8.94656]
    
    target_extent_mc_cartopy = trasform_cartopy_extent(extent_pc, pc, mc)
    target_extent_mc_pyresample = cartopy_pyresample_toggle_extent(target_extent_mc_cartopy)
    
    roi_rads = goes_2_roi(myFile,
               target_extent_mc_pyresample,
               400,1000,
               mc)
    ####
    full_filename = op.join(pathOut,ncFile[:-3])
    np.save(full_filename,roi_rads)
    myFile.close()
    return

In [ ]:
__name__ = '!__main__'
if __name__ == '__main__':#creates npy resampled image data from satellite data
    from pathlib import Path
    newPath = '/sharedData/scratch/'
    path = "G16_G17_url_list/"
    dirs = os.listdir( path )
    for txtFile in dirs:
        txtFileName = txtFile[:-4]
        SS = txtFileName[1:3]
        day = str(int(txtFileName[12:14]) + 90).zfill(3)
        match = 'M6C08_G' + SS + '_s2019' + day

        subDir = op.join(newPath, 'all_npy3')

        data = Path ( newPath )
        res = [f for f in os.listdir(data) if re.search(match,f)]
        
        with open('log', 'a') as log:
            for nc in res:
                testNC = nc[:-3] + '.npy'
                if Path(op.join(subDir,testNC)).exists() == False:#refactor to check if file already downloaded       
                    try:
                        create_nc_Numpy(nc,newPath,subDir)#names and creates file
                    except OSError as e:
                        print(e)
                        print(nc, file=log)
                        print(e, file=log)
                else:
                    continue
            
                

In [ ]:
__name__ = '__main__'
if __name__ == '__main__':#creates npy resampled image data from CLEAR SKY MASK data
    from pathlib import Path
    fromPath = '/sharedData/scratch/ACMFiles/'
    toPath = '/sharedData/scratch/all_npy3/'
    dirs = os.listdir( fromPath )
    for txtFile in dirs:
        txtFileName = txtFile[:-3]
        downloadName = txtFileName + '.npy'

        with open('log', 'a') as log:    
            if Path(op.join(toPath, downloadName)).exists()==False:
                try:
                    createACM_Numpy(txtFile,fromPath,toPath)#names and creates file
                except OSError as e:
                    print(e)
                    print(nc, file=log)
                    print(e, file=log)
            else:
                continue


In [ ]:
def createACM_Numpy(ncFile, pathIn, pathOut):
    acmList = os.listdir( pathIn )
    for acm in acmList:
        aFile = xr.open_dataset(op.join(pathIn,acm))
        clearSkyMask = aFile['BCM'].data
        lat_min = int(len(clearSkyMask)/2 -1000/2)#-500
        lat_max = int(len(clearSkyMask)/2 +1000/2)#+500
        lon_min = int(len(clearSkyMask[0])/2 -400/2)#-200
        lon_max = int(len(clearSkyMask[0])/2 +400/2)#+200

        lats = clearSkyMask[lat_min:lat_max]
        roi_CSM = []
        for row in lats:
            roi_CSM.append(list(row[lon_min:lon_max]))
        full_filename = op.join(pathOut,acm[:-3])
        np.save(full_filename,roi_CSM)
        aFile.close()
    return